In [96]:
import requests
import pandas as pd
from openai import OpenAI
import re

In [110]:
#request to LLM model
def ask_solar(content, question):
    client = OpenAI(
        api_key="HKkatmQoQ0I4t8Pt9V1xKQDNvPiUNL43",
        base_url="https://api.upstage.ai/v1/solar"
    )

    response = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": "Answer the following question:"
                + question
                + "by using the following text:"
                + content
            }
        ]
    )
    return response.choices[0].message.content

In [98]:
#downloads pdf from the url
def download_pdf(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

In [122]:
#get the list of pdf for the given article name
def get_pdf(prompt, filename):
    
    
    pdf_links = []
    iteration = 0
    while not(pdf_links):
        
        url = "https://scholar.google.com/scholar?start=" + str(iteration*10) + \
               "&hl=en&as_sdt=0%2C5&q=" + '+'.join(prompt.split())
        
        response = requests.post(url)

        pattern = r'https:\/\/[^\s]+\.pdf'
        matches = re.findall(pattern, response.text)


        for link in matches:
            response = requests.post(url = link)
            if response.headers['content-type'] in ['application/pdf']:
                pdf_links.append(link)
        
        iteration += 1
        
        
    pdf_links = list(set(pdf_links))

    #URL of the PDF file (take only one for the baseline)
    pdf_url = pdf_links[0]

    # Download the PDF
    download_pdf(pdf_url, filename)

In [112]:
#parse text in the pdf file
def get_text(filename):
    
    api_key = "upstage-la-token-20240309"
    url = "https://api.upstage.ai/v1/document-ai/layout-analyzer"
    headers = {"Authorization": f"Bearer {api_key}"}
    files = {"document": open(filename, "rb")}
    
    response = requests.post(url, headers=headers, files=files)
    
    text = ''
    for element in response.json()['elements']:
        if element['category'] == 'paragraph':
            text+= element['text']
            
    return text

In [123]:
#ready to run preset for functions
filename = 'article.pdf'
question = 'Describe all the main point and insights of given article.'

In [124]:
#Name the article here and run
#4096 tokens limitations so far
article = 'find the coaching in criticism'

get_pdf(prompt = article, filename = filename)
article_content = get_text(filename)
ask_solar(article_content[:15000], question = question)

"The article discusses the importance of managing your own feedback and how to receive it well. It highlights that feedback is crucial for personal and professional growth, but it can also be difficult to receive. The article suggests that people have three types of triggers when receiving feedback: truth triggers, relationship triggers, and identity triggers. The article also provides six steps to becoming a better receiver of feedback, including understanding your own triggers, separating the message from the messenger, and sorting feedback into evaluative or coaching categories. The article emphasizes that feedback is a process of sorting and filtering, and it requires understanding the other person's point of view, trying on new ideas, and experimenting with different approaches."

Solar LLM:
1) 4096 tokens limit; (cut is not an option)

General:
1) Compare several PDFs on 1 topic. Show mutual and contradicting parts